In [152]:
'''
Title:       Optimale Steuerung und Regelung:
Subttitle:   1. Aufgabe
Author:      Stefan Kaufmann
MaNr.        51867606
Date:        01.04.2023
'''

'\nTitle:       Optimale Steuerung und Regelung:\nSubttitle:   1. Aufgabe\nAuthor:      Stefan Kaufmann\nMaNr.        51867606\nDate:        01.04.2023\n'

# Optimale Steuerung und Regelung
## 1. Übung
### Stefan Kaufmann - 51867606

In [153]:
import numpy as np
import scipy as sp
import scipy.linalg as la
import matplotlib.pyplot as plt

mathematisches Pendel einer oszillierenden Punktmasse mit PID Regler    
$m \ddot{y}(t) + \omega^{2}y(t) = u(t)  \hspace{2cm} y(0) = 1, \hspace{1cm} \dot{y} = 0 \\
u(t) = K_{p}y(t) + K_{D}\dot{y}(t) + K_{I} \int_{0}^{t}y(\tau) d\tau 
$  
mit dem Kostenfunktional    
$
F(k,y(k;t)) = \frac{1}{2} \int_{0}^{\inf} (x^{T}(t)Qx(t) + ru^{2})dt
$


## a) Überführung in ein satisches Problem

$ x = 
\begin{bmatrix}
y & \dot{y} & \int_{0}^{t}y(\tau) d\tau 
\end{bmatrix} ^{T}
$

$
\dot{x} = 
\begin{bmatrix}
0 & 1 & 0  \\
\frac{-\omega^{2}}{m} & 0 & 0 \\
1 & 0 & 0
\end{bmatrix} x
+
\begin{bmatrix}
0 \\ \frac{1}{m} \\ 0
\end{bmatrix} u
$     
$
y = 
\begin{bmatrix}
1 & 0 & 0
\end{bmatrix} + 0u
$

In [154]:
# Parameter
m  = 1
w  = 2
Q = la.block_diag(3,4,5)
R = 6

In [155]:
# State Space System
A = np.array([[0,       1, 0],
               [-w**2/m, 0, 0],
               [1,       0, 0]])

B = np.array([0,1/m,0])
B_ = np.array([[0,1/m,0]])

C = np.array([1,0,0])
D = 0

In [184]:
N = 5000
nx = 3


def rollout(k,x):
    u     = k@x    
    dx     = A@x + B*u    
    return x+dx*0.01 , u

def cost(k):    
    x0 = np.array([1,0,0])   
    
    x_new = np.zeros((nx,N))
    u = np.zeros((1,N))
    
    x_new[:,1], u[:,0] = rollout(k,x0)  

    cost_ = x0@Q@x0*0 + R*u[:,0]**2*0
   
    for i in range(1,N-2): 
        # laufende Kosten
        x_new[:,i+1], u[:,i] = rollout(k,x_new[:,i])       
        
        cost_ += x_new[:,i+1]@Q@x_new[:,i+1]
        cost_ += u[:,i]**2*R
   
    return cost_   

print(cost([0,0,0]))   



[162759.7876639]


In [162]:
import scipy.linalg as la
P = la.solve_continuous_are(A,B_.T,Q,R)
K = -B@P/R
print(K)

[-0.30926715 -1.13366704 -0.91287093]


In [185]:

from scipy.optimize import minimize
k0 = np.array([0, -1, -1])
res = minimize(cost, k0, method="nelder-mead", options={"disp": True})
print(res)
print(cost(res.x))